In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
chroma_rar_path = '/content/drive/MyDrive/chroma.rar'
datasets_rar_path = '/content/drive/MyDrive/datasets.rar'
!unrar x {chroma_rar_path} /content/
!unrar x {datasets_rar_path} /content/



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/chroma.rar

Creating    /content/chroma                                           OK
Extracting  /content/chroma/chroma.sqlite3                                97% 99%  OK 
All OK

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/datasets.rar

Creating    /content/datasets                                         OK
Extracting  /content/datasets/business_law.txt                            12%  OK 
Extracting  /content/datasets/check_issuance_law.txt                      13%  OK 
Extracting  /content/datasets/civil_law.txt                               27%  OK 
Extracting  /content/datasets/criminal_procedure_law.txt                  41%  OK 
Extracting  /content/datasets/dispute_resolution_councils_law.txt         42%  OK 
Extracting  /content/datasets/f

In [2]:
!pip install -q accelerate langchain langchain_community unstructured sentence-transformers chromadb gradio openai langchain-openai tqdm ragas fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 7.7 

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil
import torch
import openai
from getpass import getpass
import re

In [ ]:
openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Build the Vector DB

In [5]:
%%capture
CHROMA_PATH = "./chroma"
DATA_PATH = "./datasets"

embeding_model_name = "sentence-transformers/LaBSE"
embeding_model_kwargs = {'device': device}
embeding_encode_kwargs = {'normalize_embeddings': False}

embedding_function = HuggingFaceEmbeddings(model_name=embeding_model_name,model_kwargs=embeding_model_kwargs,encode_kwargs=embeding_encode_kwargs)

In [38]:
from typing import Any, Dict, List, Optional
import numpy as np
from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, Extra, Field
from gensim import matutils

class FasttextEmbeddings(BaseModel, Embeddings):
    """Gensim sentence_transformers embedding models.
    """

    def __init__(self, model_path, **kwargs):
        """Initialize the sentence_transformer."""
        super().__init__(**kwargs)
        try:
            import fasttext

        except ImportError as exc:
            raise ImportError(
                "Could not import gensim python package. "
                "Please install it with `pip install gensim`."
            ) from exc

        self.vectorStore = fasttext.load_model(model_path)

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.allow

    def get_vector_embedding(self,word):
        vec = self.vectorStore[word]
        vec = matutils.unitvec(np.array(vec))
        return vec

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a HuggingFace transformer model.
        Args:
            texts: The list of texts to embed.
        Returns:
            List of embeddings, one for each text.
        """

        texts = list(map(lambda x: x.replace("\n", " "), texts))
        embeddings = []
        for text in texts:
            vector =  self.get_vector_embedding(text)
            embeddings.append(vector.tolist())
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a HuggingFace transformer model.
        Args:
            text: The text to embed.
        Returns:
            Embeddings for the text.
        """
        return self.embed_documents([text])[0]

In [39]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-fa-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.words


len(model.words)

x = model['کیک']
print(len(x))
model['کیک']

model.bin:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

300


array([ 1.80795282e-01,  2.89481640e-01,  1.45641848e-01, -1.46620259e-01,
        4.71592918e-02,  1.05648935e-01,  1.21546060e-01,  1.28660098e-01,
        1.85307294e-01,  3.62608969e-01,  2.16912776e-02, -1.40143529e-01,
       -3.92908007e-02, -8.84452164e-02, -1.02125965e-01,  8.76953453e-02,
        2.05178168e-02, -2.73209438e-02, -2.65041869e-02, -6.31793067e-02,
       -1.11681864e-01, -5.82233407e-02,  6.12472259e-02, -8.65362361e-02,
        5.42299822e-02, -1.10495016e-01, -2.01638162e-01, -1.41403228e-02,
       -1.21957362e-01, -1.00320533e-01, -8.35162997e-02,  3.26706655e-02,
       -5.16331755e-02,  1.38361827e-01, -1.23719230e-01, -6.26623631e-03,
        5.25223278e-02,  2.08721533e-02, -1.53537646e-01, -1.26743376e-01,
        1.26412153e-01, -1.87244326e-01,  4.00330983e-02,  1.84700742e-01,
        8.92394111e-02, -2.30759323e-01,  4.38205525e-02,  7.11347163e-02,
        8.31701830e-02, -2.55037323e-02, -1.08600587e-01,  1.15021810e-01,
        3.48614268e-02,  

In [41]:
del model

In [42]:
embeding_encode_kwargs = {'normalize_embeddings': False}
embedding_function = FasttextEmbeddings(model_path)


## Naive Document Splitting

In [ ]:
with open('/content/business_law.txt', 'r') as file:
  text = file.read()

FileNotFoundError: [Errno 2] No such file or directory: '/content/business_law.txt'

In [ ]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.txt")
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    return chunks


def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(
        chunks,
        embedding_function,
        persist_directory=CHROMA_PATH
    )

    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [ ]:
generate_data_store()

In [ ]:
# backup the sqlite database

In [22]:
# @title Add Madde Column
import pandas as pd
import re

!cp -r /content/drive/MyDrive/IRI_LAW/evaluation ./evaluation

BUSSINESS_LAW_PATH = './evaluation/business_law/easy_new.csv'
df = pd.read_csv(BUSSINESS_LAW_PATH)
print(df.head())

pattern = r'\d+'

def extract_madde_numbers(answer):
    parts = answer.split(".")
    last_part = parts[-1]
    madde_numbers = re.findall(pattern, last_part)
    return madde_numbers

df['Madde'] = df['Answer'].apply(extract_madde_numbers)
df.to_csv('/content/drive/MyDrive/IRI_LAW/evaluation/business_law/easy_new.csv', index=False)

df


                                            Question  \
0                              شرایط تاجر بودن چیست؟   
1  خرید یا تحصیل مال منقول برای چه منظوری جزء معا...   
2  انواع عملیاتی که جزء معاملات تجارتی محسوب می‌ش...   
3  تأسیس و به کار انداختن کارخانه تحت چه شرایطی ج...   
4  چه نوع معاملاتی بین تجار و کسبه تجارتی محسوب م...   

                                              Answer  Madde  
0  تاجر کسی است که شغل معمولی خود را معاملات تجار...  ['1']  
1  برای فروش یا اجاره، چه تصرفاتی در آن شده باشد ...  ['2']  
2  عملیات دلالی، حق‌العمل‌کاری، عاملی و تأسیسات م...  ['2']  
3                برای رفع حوائج شخصی نباشد. (ماده 2)  ['2']  
4  کلیه معاملات بین تجار و کسبه و صرافان و بانک‌ه...  ['3']  


,Question,Answer,Madde
0,شرایط تاجر بودن چیست؟,تاجر کسی است که شغل معمولی خود را معاملات تجار...,[1]
1,خرید یا تحصیل مال منقول برای چه منظوری جزء معا...,برای فروش یا اجاره، چه تصرفاتی در آن شده باشد ...,[2]
2,انواع عملیاتی که جزء معاملات تجارتی محسوب می‌ش...,عملیات دلالی، حق‌العمل‌کاری، عاملی و تأسیسات م...,[2]
3,تأسیس و به کار انداختن کارخانه تحت چه شرایطی ج...,برای رفع حوائج شخصی نباشد. (ماده 2),[2]
4,چه نوع معاملاتی بین تجار و کسبه تجارتی محسوب م...,کلیه معاملات بین تجار و کسبه و صرافان و بانک‌ه...,[3]
5,دفاتری که تاجر باید داشته باشد کدامند؟,دفتر روزنامه، دفتر کل، دفتر دارایی، و دفتر کپی...,[6]
6,دفتر روزنامه چه نوع اطلاعاتی را باید ثبت کند؟,مطالبات، دیون، داد و ستد تجارتی و معاملات راجع...,[7]
7,دفتر کل چگونه باید تنظیم شود؟,کلیه معاملات را لااقل هفته یک مرتبه از دفتر رو...,[8]
8,دفتر دارایی چه اطلاعاتی را شامل می‌شود؟,صورت جامعی از کلیه دارایی منقول و غیر منقول و ...,[9]
9,نحوه ثبت مراسلات و مخابرات در دفتر کپیه چگونه ...,کلیه مراسلات و مخابرات و صورت‌حساب‌های صادره ب...,[10]


## Chunking based on Law Articles

In [8]:
BUSINESS_LAW_TXT_PATH = '/content/datasets/business_law.txt'

with open(BUSINESS_LAW_TXT_PATH, 'r') as file:
  text = file.read()

# preprocess
temp = ''
for c in text:
  if c != '\u200c':
    temp += c

text = temp

In [9]:
pattern = r'(^ماده) (\d+)'

matches = re.finditer(pattern, text, re.MULTILINE)
indexes = [m.start() for m in matches]

splitted = []
for i, j in zip([0] + indexes, indexes + [len(text)]):
  splitted.append(text[i:j])

In [10]:
from langchain.docstore.document import Document

chunks = []
for d in splitted:
  chunks.append(Document(page_content=d))

In [43]:
db = Chroma.from_documents(
    chunks,
    embedding_function,
    persist_directory='/content/chroma_fasttext'
)
db.persist()

In [ ]:
question = 'شرایط تاجر بودن چیست؟'
r = db.similarity_search_with_relevance_scores(question, k=5)
r

[(Document(page_content='ماده 190(الحاقي 24ˏ12ˏ1347)- پيشنهاد هيأت مديره راجع بكاهش سرمايه بايد حداقل چهل و پنج روز قبل از تشكيل مجمع عمومي فوقالعاده ببازرس يا بازرسان شركت تسليم گردد.\nپيشنهاد مزبور بايد متضمن توجيه لزوم كاهش سرمايه و همچنين شامل گزارشي درباره امور شركت از بدو سال مالي در جريان و اگر تا آنموقع مجمععمومي نسبت بحسابهاي سال مالي قبل تصميم نگرفته باشد حاكي از وضع شركت از ابتداي سال مالي قبل باشد.\n\n'),
  0.5427099811109837),
 (Document(page_content='ماده 233(الحاقي 24ˏ12ˏ1347)- در تنظيم حساب عملكرد و حساب سود و زيان و ترازنامه شركت بايد همان شكل و روشهاي ارزيابي كه در سال مالي قبل از آن بكاررفته است رعايت شود. با وجود اين در صورتيكه تغييري در شكل و روشهاي ارزيابي سال قبل از آن مورد نظر باشد بايد اسناد مذكور بهر دو شكل وهر دو روش ارزيابي تنظيم گردد تا مجمع عمومي با ملاحظه آنها و با توجه بگزارش هيأت مديره و بازرسان نسبت به تغييرات پيشنهادي تصميم بگيرد.\n\n'),
  0.5359848613751914),
 (Document(page_content='ماده 168 - اعضاءِ هيئت نظار دفاتر و صندوق و كليه اسناد شركت را تحت 

In [ ]:
# db.persist()

In [ ]:
# backup the sqlite database

# Calculate MRR

In [48]:
import pandas as pd
!cp -r /content/drive/MyDrive/IRI_LAW/evaluation ./evaluation

BUSSINESS_LAW_PATH = '/content/drive/MyDrive/IRI_LAW/evaluation/business_law/hard_new.csv'
df = pd.read_csv(BUSSINESS_LAW_PATH)
df.head()

,Question,Answer,Madde
0,چه شرایطی برای تشکیل و ثبت یک شرکت با مسئولیت ...,برای تشکیل یک شرکت با مسئولیت محدود نیاز به پر...,"['96', '97', '111']"
1,در یک شرکت با مسئولیت محدود، چگونه مدیریت انجا...,مدیریت شرکت با مسئولیت محدود توسط یک یا چند مد...,"['104', '105']"
2,چه فرآیندی باید برای انحلال یک شرکت با مسئولیت...,انحلال شرکت ممکن است در صورت تصمیم شرکاء یا بر...,"['114', '101']"
3,تفاوت‌های اصلی بین شرکت‌های با مسئولیت محدود و...,در شرکت با مسئولیت محدود، مدیریت می‌تواند توسط...,"['104', '102', '123', '116']"
4,چگونه می‌توان یک شرکت تضامنی را به شرکت سهامی ...,تبدیل یک شرکت تضامنی به شرکت سهامی نیاز به تصو...,['135']


In [24]:
!cp -r /content/drive/MyDrive/IRI_LAW/chroma ./chroma
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [49]:
predicted_madde_list = []
rr_list = []

import ast

if df['Madde'].apply(lambda x: isinstance(x, str)).all():
  df['Madde'] = df['Madde'].apply(ast.literal_eval)
  df['Madde'] = df['Madde'].apply(lambda x: [int(num) for num in x])



for question in df['Question']:
    retrieval_results = db.similarity_search_with_relevance_scores(question, k=5)

    predicted_madde = []
    rr = 0
    madde_actual = df.loc[df['Question'] == question, 'Madde'].values[0]

    for rank, result in enumerate(retrieval_results, 1):
        if 'ماده' in result[0].page_content:
            madde_number = result[0].page_content.split('ماده ')[1].split(' ')[0]
            madde_number = re.search(r'\d+', madde_number).group()
            madde_number = int(madde_number)
            predicted_madde.append(madde_number)

            if madde_number in madde_actual:
                rr = 1 / rank
                break

    rr_list.append(rr)
    predicted_madde_list.append(predicted_madde)


mrr = sum(rr_list) / len(rr_list) if len(rr_list) > 0 else 0

df['Predicted_Madde'] = predicted_madde_list
df['MRR'] = rr_list

average_row = {'Predicted_Madde': 'Average MRR', 'MRR': mrr}
df = df.append(average_row, ignore_index=True)

print("Mean Reciprocal Rank (MRR):", mrr)
df.to_csv('/content/drive/MyDrive/IRI_LAW/results/MRR_fasttext_hard.csv', index=False)
df

Mean Reciprocal Rank (MRR): 0.10714285714285714


<ipython-input-49-eef4ba9fa163>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(average_row, ignore_index=True)


,Question,Answer,Madde,Predicted_Madde,MRR
0,چه شرایطی برای تشکیل و ثبت یک شرکت با مسئولیت ...,برای تشکیل یک شرکت با مسئولیت محدود نیاز به پر...,"[96, 97, 111]","[241, 9, 170, 169, 141]",0.000000
1,در یک شرکت با مسئولیت محدود، چگونه مدیریت انجا...,مدیریت شرکت با مسئولیت محدود توسط یک یا چند مد...,"[104, 105]","[154, 141, 153, 247, 150]",0.000000
2,چه فرآیندی باید برای انحلال یک شرکت با مسئولیت...,انحلال شرکت ممکن است در صورت تصمیم شرکاء یا بر...,"[114, 101]","[154, 129, 141, 153, 279]",0.000000
3,تفاوت‌های اصلی بین شرکت‌های با مسئولیت محدود و...,در شرکت با مسئولیت محدود، مدیریت می‌تواند توسط...,"[104, 102, 123, 116]","[154, 141, 153, 298, 247]",0.000000
4,چگونه می‌توان یک شرکت تضامنی را به شرکت سهامی ...,تبدیل یک شرکت تضامنی به شرکت سهامی نیاز به تصو...,[135],"[104, 241, 12, 251, 9]",0.000000
5,در چه شرایطی یک شرکت با مسئولیت محدود می‌تواند...,سهم‌الشرکه‌های غیر نقدی باید هنگام تشکیل شرکت ...,"[96, 98]","[154, 141, 298, 247, 241]",0.000000
6,چه مواردی موجب بطلان یک شرکت با مسئولیت محدود ...,تشکیل شرکت برخلاف مواد 96 و 97 موجب بطلان شرکت...,"[100, 101]","[154, 141, 150, 153, 109]",0.000000
7,در صورت فوت یا محجوریت یکی از شرکاء شرکت تضامن...,در صورت فوت یا محجوریت یکی از شرکاء، بقاء شرکت...,"[139, 140]","[80, 293, 542, 9, 268]",0.000000
8,چه شرایطی برای انتقال سهم‌الشرکه در شرکت‌های ت...,در شرکت‌های با مسئولیت محدود، انتقال سهم‌الشرک...,"[102, 103, 123]","[174, 298, 9, 294, 247]",0.000000
9,فرآیند اتخاذ تصمیمات در شرکت‌های با مسئولیت مح...,در شرکت‌های با مسئولیت محدود، تصمیمات باید با ...,"[106, 136]","[145, 118, 170, 270, 154]",0.000000


# Load the Vector DB

In [ ]:
# vector database for business law
!cp -r /content/drive/MyDrive/IRI_LAW/chroma ./chroma

In [ ]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [ ]:
def find_relevant_results(query, k=3):
  results = db.similarity_search_with_relevance_scores(query, k=k)
  return results

In [ ]:
res = find_relevant_results('چگونه می‌توان شرکتی را به ثبت رساند و چه مدارکی لازم است؟')
res

[(Document(page_content='امضاي مجاز شركت رسيده و مشتمل بر نكات زير باشد: الف - نام و شماره ثبت شركت. ب - موضوع شركت و نوع فعاليتهاي آن ج - مركز اصلي شركت و در صورتيكه شركت شعبي داشته باشد نشاني شعب آن د - در صورتيكه شركت براي مدت محدود تشكيل شده باشد تاريخ انقضاء مدت آن هـ - سرمايه شركت و مبلغ پرداخت شده آن. و - اگر سهام', metadata={'source': 'datasets/business_law.txt', 'start_index': 108869}),
  0.40191792789816005),
 (Document(page_content='ماده 8(الحاقي 24ˏ12ˏ1347)- طرح اساسنامه بايد با قيد تاريخ به امضاء مؤسسين رسيده و مشتمل بر مطالب زير باشد: 1 - نام شركت 2 - موضوع شركت بطور صريح و منجز 3 - مدت شركت 4 - مركز اصلي شركت و محل شعب آن اگر تأسيس شعبه مورد نظر باشد. 5 - مبلغ سرمايه شركت و تعيين مقدار نقد و غير نقد آن به تفكيك. 6 - تعداد', metadata={'source': 'datasets/business_law.txt', 'start_index': 8311}),
  0.3983542432691699),
 (Document(page_content='ماده 60(الحاقي 24ˏ12ˏ1347)- ورقه قرضه بايد شامل نكات زير بوده و بهمان ترتيبي كه براي امضاي اوراق سهام مقرر شده است امضاء بشود:\n\n1

In [ ]:
print(res[0][0].page_content)

امضاي مجاز شركت رسيده و مشتمل بر نكات زير باشد: الف - نام و شماره ثبت شركت. ب - موضوع شركت و نوع فعاليتهاي آن ج - مركز اصلي شركت و در صورتيكه شركت شعبي داشته باشد نشاني شعب آن د - در صورتيكه شركت براي مدت محدود تشكيل شده باشد تاريخ انقضاء مدت آن هـ - سرمايه شركت و مبلغ پرداخت شده آن. و - اگر سهام


# Configure RAG



## Init Language Model

## MaralGPT-7B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name_or_id = "MaralGPT/Maral-7B-alpha-1"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)
llm = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
from typing import Any, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM


class CustomLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "MistralForCausalLM"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
      inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
      outputs = llm.generate(**inputs, generation_config=generation_config)
      return tokenizer.decode(outputs[0], skip_special_tokens=True)

llm = CustomLLM()

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.5,
    max_new_tokens=300,
    pad_token_id=tokenizer.eos_token_id
)
def process_query(query):
    results = find_relevant_results(query)
    if len(results) == 0 or results[0][1] < 0.25:
        return ("اطلاعاتی که مرتبط با سوال شما باشد را در پایگاه دانش خود پیدا نکردم!")
    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt = prompt.format(context=context, question=query)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = llm.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## GPT 3.5-turbu

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## Create Prompt Template

In [ ]:
from langchain.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
فقط بر اساس متن زیر به سوال پاسخ دهید:

{context}

---

با توجه به متن بالا به سوال پاسخ دهید:
{question}

پاسخ:
"""

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

## Chain All Elements

In [ ]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .25, "k": 3})

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

# Process a Single Question

## Serve in Gradio

In [ ]:
import gradio as gr

def run_query(query):
  resp = qa_chain.invoke({"question" : query})
  return resp["response"].content

with gr.Blocks() as demo:
    query = gr.Textbox(label="Question", rtl=True, lines=5)
    output = gr.Textbox(label="Answer", rtl=True, lines=10)
    greet_btn = gr.Button("Query")
    greet_btn.click(fn=run_query, inputs=query, outputs=output, api_name="run_query")

demo.launch();

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3e9c6ba962cf00ac5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
demo.close()

Closing server running on port: 7860


# Evaluation

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!cp -r /content/drive/MyDrive/IRI_LAW/evaluation ./evaluation

In [ ]:
import pandas as pd

BUSSINESS_LAW_PATH = './evaluation/business_law/easy.csv'
df = pd.read_csv(BUSSINESS_LAW_PATH)
df.head()

,Question,Answer
0,شرایط تاجر بودن چیست؟,تاجر کسی است که شغل معمولی خود را معاملات تجار...
1,خرید یا تحصیل مال منقول برای چه منظوری جزء معا...,برای فروش یا اجاره، چه تصرفاتی در آن شده باشد ...
2,انواع عملیاتی که جزء معاملات تجارتی محسوب می‌ش...,عملیات دلالی، حق‌العمل‌کاری، عاملی و تأسیسات م...
3,تأسیس و به کار انداختن کارخانه تحت چه شرایطی ج...,برای رفع حوائج شخصی نباشد. (ماده 2)
4,چه نوع معاملاتی بین تجار و کسبه تجارتی محسوب م...,کلیه معاملات بین تجار و کسبه و صرافان و بانک‌ه...


In [ ]:
test_questions = df["Question"].values.tolist()
test_groundtruths = df["Answer"].values.tolist()

In [ ]:
from tqdm import tqdm

answers = []
contexts = []

for question in tqdm(test_questions, desc='Processing'):
  response = qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Processing:  35%|███▌      | 7/20 [00:12<00:23,  1.80s/it]/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:343: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.25
  warnings.warn(
Processing:  65%|██████▌   | 13/20 [00:23<00:13,  1.96s/it]/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:343: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.25
  warnings.warn(
Processing:  90%|█████████ | 18/20 [00:32<00:04,  2.22s/it]/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:343: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.25
  warnings.warn(
Processing: 100%|██████████| 20/20 [00:36<00:00,  1.81s/it]


In [ ]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [ ]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
results

{'faithfulness': 0.7367, 'answer_relevancy': 0.8038, 'context_recall': 0.8108, 'context_precision': 0.5458, 'answer_correctness': 0.6188}

In [ ]:
res_pd = results.to_pandas()
res_pd

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,شرایط تاجر بودن چیست؟,شرایط تاجر بودن شامل انجام معاملات تجاری به صو...,[ماده 1 - تاجر كسي است كه شغل معمولي خود را مع...,تاجر کسی است که شغل معمولی خود را معاملات تجار...,0.00,0.966249,1.000000,1.000000,0.895567
1,خرید یا تحصیل مال منقول برای چه منظوری جزء معا...,خرید یا تحصیل مال منقول برای منظور فروش یا اجا...,[ماده 4 - معاملات غير منقول بهيچوجه تجارتي محس...,برای فروش یا اجاره، چه تصرفاتی در آن شده باشد ...,1.00,0.918036,1.000000,0.333333,0.728639
2,انواع عملیاتی که جزء معاملات تجارتی محسوب می‌ش...,همه معاملات تجارتی محسوب می‌شوند مگر اینکه ثاب...,[ماده 4 - معاملات غير منقول بهيچوجه تجارتي محس...,عملیات دلالی، حق‌العمل‌کاری، عاملی و تأسیسات م...,0.50,0.892479,1.000000,0.333333,0.857541
3,تأسیس و به کار انداختن کارخانه تحت چه شرایطی ج...,تأسیس و به کار انداختن کارخانه جزء معاملات تجا...,[ماده 1 - تاجر كسي است كه شغل معمولي خود را مع...,برای رفع حوائج شخصی نباشد. (ماده 2),0.00,0.926233,0.000000,0.000000,0.706671
4,چه نوع معاملاتی بین تجار و کسبه تجارتی محسوب م...,کلیه معاملات بین تجار و کسبه تجارتی محسوب می‌شود.,[ماده 3 - معاملات ذيل باعتبار تاجر بودن متعامل...,کلیه معاملات بین تجار و کسبه و صرافان و بانک‌ه...,NaN,0.945362,1.000000,1.000000,0.737230
5,دفاتری که تاجر باید داشته باشد کدامند؟,1. دفتر روزنامه\n2. دفتر کپیه,[ماده 438 - انبارها و حجرهها و صندوق و اسناد و...,دفتر روزنامه، دفتر کل، دفتر دارایی، و دفتر کپی...,1.00,0.748576,0.750000,1.000000,0.732888
6,دفتر روزنامه چه نوع اطلاعاتی را باید ثبت کند؟,دفتر روزنامه باید همه روزه مطالبات و دیون و دا...,[ماده 7 - دفتر روزنامه دفتري است كه تاجر بايد ...,مطالبات، دیون، داد و ستد تجارتی و معاملات راجع...,1.00,0.811721,1.000000,1.000000,0.728294
7,دفتر کل چگونه باید تنظیم شود؟,باید تنظیم شود.,[],کلیه معاملات را لااقل هفته یک مرتبه از دفتر رو...,NaN,0.000000,1.000000,0.000000,0.205245
8,دفتر دارایی چه اطلاعاتی را شامل می‌شود؟,دفتر دارایی شامل اسناد، نوشتجات، اسباب و اثاثی...,[ماده 438 - انبارها و حجرهها و صندوق و اسناد و...,صورت جامعی از کلیه دارایی منقول و غیر منقول و ...,1.00,0.950056,0.000000,0.000000,0.213697
9,نحوه ثبت مراسلات و مخابرات در دفتر کپیه چگونه ...,مراسلات و مخابرات باید توسط تاجر در دفتر کپیه ...,[ماده 10 - دفتر كپيه دفتري است كه تاجر بايد كل...,کلیه مراسلات و مخابرات و صورت‌حساب‌های صادره ب...,1.00,0.901492,1.000000,1.000000,0.733164


In [ ]:
res_pd.to_csv('results_article_based.csv', index=True)